In [ ]:
# %matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.artist import Artist
import matplotlib as MPL
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle

from IPython.display import HTML

import numpy as np
from astropy.io import fits
from glob import glob
from datetime import datetime
import subprocess
from io import StringIO
from dataclasses import dataclass


In [ ]:
fitsFiles = set(glob("pcoWideSourceTest_01_20220709_*.fits")) - set(glob("*.xy.*"))
fileTimes = np.ndarray((len(fitsFiles), 1), dtype=datetime)

for idx, name in enumerate(fitsFiles):
    ns = name.split('_')
    ds = ns[2] + ns[3]
    ft = datetime.strptime(ds, "%Y%m%d%H%M%S.%f.fits")
    
    fileTimes[idx] = ft

sortedFileNames = np.asanyarray([x for _,x in sorted(zip(fileTimes, fitsFiles))])
# print(sortedFileNames)

### Loop over N number of frames.
#### Store data, and source bboxes from image2xy and TSimag2xy

In [ ]:
#==========================#
# set number of frames here
#==========================#
numFrames = 100

@dataclass
class xylist:
    sources: np.ndarray
    
    def __post_init__(self):
        
        self.sortedSources = self.sources[self.sources[:,2].argsort()[::-1]] # sort by flux
    
    def topNsources(self, numSources):
        return self.sortedSources[0:numSources, :]
    
def get_libtsSources(fname):
    
    tsLibTestCmd = "/home/viliam/thaiSpice/image2xyTesting/build/./testLib"
    procRes = subprocess.run([tsLibTestCmd, fname], capture_output=True, text=True)
    
    # print(type(procRes.stdout))
    data = np.loadtxt(StringIO(procRes.stdout), delimiter=',', dtype=float)
    return xylist(data)
    # sortedByFlux = data[data[:,2].argsort()]

#==========================#
# main loop
#==========================#

ims = []
xyIms = []

for idx, fname in enumerate(sortedFileNames):
    
    print(f"Processing frame #: {idx:04d}", end='\r')
    
    with fits.open(fname) as img:
        imgData = img[0].data
        imgData = imgStretchDisplay(imgData)
        
        xy = get_libtsSources(fname)

        ims.append(imgData)
        xyIms.append(xy)
        
        if idx > numFrames:
            break


# ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True,
#                                 repeat_delay=1000)

### Cell below does the animation

In [ ]:
MPL.rcParams['animation.embed_limit'] = 100

#==========================#
def imgStretchDisplay(imgData):
    
    md = np.mean(imgData)
    idmm = np.abs(imgData - md)
    idmm = np.float_power(idmm, 0.8)
    
    return idmm
#==========================#



fig, ax = plt.subplots(figsize=(10,10))
dataDisplay = ax.imshow(ims[0], vmax=50, vmin=0, animated=True);

# print(xyIms[0].topNsources(4))
# for xx in xyIms[0].topNsources(6):
#     print(xx[0:2])
boxSize = 80
bs2 = int(boxSize/2)

sBxs = [Rectangle((xx[0]-bs2, xx[1]-bs2), boxSize, boxSize) for xx in xyIms[0].topNsources(4)]
pc = PatchCollection(sBxs, facecolor='none', edgecolor='C1')
pcax = ax.add_collection(pc)

txtArtist = ax.text(50, 1000, f"Frame number: {idx:03d}",
            fontsize=15,
            color='white')
            
def animFrames(idx):
    
    global pcax
    
    dataDisplay.set_data(ims[idx])
    txtArtist.set_text(f"Frame number: {idx:03d}")
    
    # pcax.remove()
    
    # for rr in sBxs:
    #     rr.remove()
    # sBxs[0]
    
    recArray = [Rectangle((xx[0]-bs2, xx[1]-bs2), boxSize, boxSize) for xx in xyIms[idx].topNsources(4)]
    pc = PatchCollection(recArray, facecolor='none', edgecolor='C1')
    
    pcax.remove()
    pcax = ax.add_collection(pc)
    
    # rpc.remove()
    # vvs = pc.get_paths()
    # rpc.set_paths(vvs)
    # ax.add_collection(pc)
    # rpc.set(recArray)
    # rpc.set_array(recArray)
    # rpc.set_array
    
    return (dataDisplay, )

anim = animation.FuncAnimation(fig, animFrames, frames=numFrames, interval=50, blit=False);
HTML(anim.to_jshtml())